In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

In [2]:
train = pd.read_csv('data/train_dataset.csv')
train

,time,JS_NH3,CS_NH3,JS_TN,CS_TN,JS_LL,CS_LL,MCCS_NH4,MCCS_NO3,JS_COD,CS_COD,JS_SW,CS_SW,B_HYC_NH4,B_HYC_XD,B_HYC_MLSS,B_HYC_JS_DO,B_HYC_DO,B_CS_MQ_SSLL,B_QY_ORP,N_HYC_NH4,N_HYC_XD,N_HYC_MLSS,N_HYC_JS_DO,N_HYC_DO,N_CS_MQ_SSLL,N_QY_ORP,Label1,Label2
0,2022/1/4 0:00,31.0164,0.0583,36.5537,10.5156,1685.3599,996.0668,0.1639,8.2715,169.3,4.0,23.8045,23.9361,0.0000,0.0000,6759.8159,0.4019,1.5762,4395.0459,167.3177,0.5787,4.2710,6455.5645,0.5013,2.3087,3010.5042,127.9659,NaN,NaN
1,2022/1/4 0:02,31.0164,0.0583,36.5537,10.5156,1739.0143,874.0959,0.1639,8.3518,169.3,4.0,23.8048,23.9367,0.0000,0.0000,6789.2598,0.5040,1.6747,4393.9658,167.3177,0.7004,3.3759,6410.8105,0.6029,2.2781,3009.2285,128.2552,NaN,NaN
2,2022/1/4 0:04,31.9567,0.0583,36.5537,10.5156,1729.6257,1324.8105,0.1630,8.3116,169.3,4.0,NaN,NaN,0.0000,0.0000,6801.4292,0.4310,1.5569,4396.1250,167.3177,0.6278,4.0315,6407.2769,0.4522,2.3421,3009.3267,127.9659,NaN,NaN
3,2022/1/4 0:06,31.9567,0.0583,36.5537,10.5156,1665.6812,723.9781,0.1630,8.1999,169.3,4.0,NaN,NaN,0.0000,0.0000,6778.2676,0.4326,1.5259,4395.5361,167.3177,0.6239,4.0315,6401.7803,0.7498,2.2490,3011.3877,128.8339,NaN,NaN
4,2022/1/4 0:08,31.9567,0.0583,36.5537,10.5156,1628.1198,741.2993,0.1630,8.1380,169.3,4.0,NaN,NaN,0.0000,0.0000,6794.7559,0.5456,1.5538,4395.0459,167.3177,0.7024,3.6841,6403.3506,0.5896,2.3268,3013.2522,128.3999,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140475,2022/7/18 2:30,27.1314,0.0169,30.2603,9.2619,985.8353,1111.9569,0.1428,9.0365,144.6,9.2,27.3868,31.1426,0.2018,7.1839,6057.0928,0.1099,1.5306,4396.0273,133.7529,1.5523,11.8792,5881.2158,0.1970,1.4034,3011.6821,122.6852,10164.06,9161.48
140476,2022/7/18 2:32,27.1314,0.0169,30.2603,9.2619,1008.5851,1680.4688,0.1425,9.0495,144.6,9.2,27.3911,31.1435,0.1999,8.6522,6098.3140,0.1197,1.4823,4393.5732,133.3188,1.5641,11.9833,5919.2964,0.1413,1.2389,3012.5654,122.1788,9323.08,8399.22
140477,2022/7/18 2:34,27.1314,0.0169,30.2603,9.2619,977.8567,1633.5309,0.1425,9.0419,144.6,9.2,27.3971,31.1448,0.2156,7.7061,6073.9736,0.1424,1.4859,4394.0645,132.9572,1.5778,12.1246,5966.4062,0.1326,1.3602,3015.1169,121.8895,11013.08,9923.92
140478,2022/7/18 2:36,27.1314,0.0169,30.2603,9.2619,974.0818,1247.8890,0.1425,9.0093,144.6,9.2,27.4090,31.1455,0.2117,9.1076,6104.5952,0.1102,1.4230,4390.8252,132.9572,1.5523,12.2600,5993.8872,0.1668,1.5616,3012.5654,121.6001,10143.46,9162.94


In [3]:
# 只有 1/4 是有标签的, 后面是要做伪标签?

train[train['Label1'].notna()].shape[0], train[train['Label1'].isna()].shape[0]

(35068, 105412)

In [4]:
test = pd.read_csv('data/evaluation_public.csv')
test

,time,JS_NH3,CS_NH3,JS_TN,CS_TN,JS_LL,CS_LL,MCCS_NH4,MCCS_NO3,JS_COD,CS_COD,JS_SW,CS_SW,B_HYC_NH4,B_HYC_XD,B_HYC_MLSS,B_HYC_JS_DO,B_HYC_DO,B_CS_MQ_SSLL,B_QY_ORP,N_HYC_NH4,N_HYC_XD,N_HYC_MLSS,N_HYC_JS_DO,N_HYC_DO,N_CS_MQ_SSLL,N_QY_ORP
0,2022/7/18 2:40,27.1314,0.0169,33.0682,9.2619,1418.9423,1714.6394,0.1425,9.1580,149.0,9.2,27.4158,31.1471,0.2117,7.2605,6091.2476,0.1126,1.3685,4392.6904,132.8125,1.5445,12.5348,5995.4580,0.2080,1.1596,3012.6638,121.0938
1,2022/7/18 2:42,27.1314,0.0169,33.0682,9.2619,1396.9487,-1936.3779,0.1425,9.1721,149.0,9.2,27.3844,31.1482,0.2097,8.2164,6085.3589,0.1503,1.2417,4394.0645,132.8125,1.5621,12.6369,5980.5396,0.1589,1.3869,3015.7061,120.8767
2,2022/7/18 2:44,27.1314,0.0169,33.0682,9.2619,620.5256,830.3297,0.1425,9.1970,149.0,9.2,27.3487,31.1494,0.2077,7.2212,6100.2769,0.0777,1.2279,4393.2793,132.8125,1.5523,12.7370,5986.0347,0.1503,1.3304,3015.0190,120.6597
3,2022/7/18 2:46,27.1314,0.0169,33.0682,9.2619,1330.6614,1313.9777,0.1425,9.0907,149.0,9.2,27.3216,31.1502,0.2077,7.8395,6085.3589,0.1448,1.4109,4396.3213,132.8125,1.5366,12.8430,5973.8652,0.1742,1.3312,3013.1543,120.5151
4,2022/7/18 2:48,27.1314,0.0169,33.0682,9.1671,1386.0891,1252.3951,0.1425,9.0234,149.0,9.2,27.2979,31.1516,0.2077,8.2105,6048.8486,0.1330,1.2676,4396.2236,132.7401,1.5327,12.9765,5973.8652,0.1499,1.1537,3015.6079,120.4427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2022/7/31 23:50,23.5977,0.0132,31.9466,9.3026,2050.2271,1440.8977,0.2193,9.0017,127.3,5.6,27.6721,31.2442,0.2117,5.9257,6733.1201,0.1295,1.0395,4394.2607,122.1788,4.3711,4.2808,6687.9736,0.1660,1.2425,3007.1675,120.2980
9996,2022/7/31 23:52,23.5977,0.0132,31.9466,9.3026,2091.4170,1291.4475,0.2193,9.0571,127.3,5.6,27.6787,31.2457,0.2156,6.0984,6690.3291,0.1012,1.0646,4392.6904,122.1065,4.2297,4.3868,6781.4082,0.1636,1.3763,3007.4619,120.2980
9997,2022/7/31 23:54,23.5977,0.0132,31.9466,9.3026,2114.0769,1369.1768,0.2193,9.2350,127.3,5.6,27.6858,31.2480,0.2018,5.8825,6468.9126,0.1919,1.2197,4394.9473,122.1788,4.1610,4.5261,6767.6680,0.1723,1.3445,3011.9766,120.2980
9998,2022/7/31 23:56,23.5977,0.0132,31.9466,9.3026,1663.0690,912.1893,0.2211,9.1753,127.3,5.6,27.6935,31.2492,0.2018,6.4635,6472.4453,0.2567,1.2220,4398.3828,122.2512,4.3789,4.2533,6685.2256,0.1546,1.3049,3012.0747,120.3704


In [5]:
# 1. 数据说明里表示，北生化池和南生化池在生产过程中不会互相影响, 可以先试下分开两部分
# 2. 只用有 label 的数据

train_B = train[[i for i in train.columns if (i != 'Label2' and not i.startswith('N_'))]].copy()
train_N = train[[i for i in train.columns if (i != 'Label1' and not i.startswith('B_'))]].copy()

train_B = train_B[train_B['Label1'].notna()].copy().reset_index(drop=True)
train_N = train_N[train_N['Label2'].notna()].copy().reset_index(drop=True)

test_B = test[[i for i in test.columns if not i.startswith('N_')]].copy()
test_N = test[[i for i in test.columns if not i.startswith('B_')]].copy()

In [6]:
# 时间特征

def add_datetime_feats(df):
    df['time'] = pd.to_datetime(df['time'])
    df['day'] = df['time'].dt.day
    df['hour'] = df['time'].dt.hour
    df['dayofweek'] = df['time'].dt.dayofweek
    
    return df

train_B = add_datetime_feats(train_B)
train_N = add_datetime_feats(train_N)
test_B = add_datetime_feats(test_B)
test_N = add_datetime_feats(test_N)

In [7]:
# 做点比率数值特征

def add_ratio_feats(df, type_='B'):
    df['JS_CS_NH3_ratio'] = df['JS_NH3'] / (df['CS_NH3'] + 1e-3)
    df['JS_CS_TN_ratio'] = df['JS_TN'] / (df['CS_TN'] + 1e-3)
    df['JS_CS_LL_ratio']  = df['JS_LL'] / (df['CS_LL'] + 1e-3)
    df['MCCS_NH4_NH3_ratio'] = df['MCCS_NH4'] / (df['CS_NH3'] + 1e-3)
    df['MCCS_NO3_NH3_ratio'] = df['MCCS_NO3'] / (df['CS_NH3'] + 1e-3)
    df['JS_CS_COD_ratio'] = df['JS_COD'] / (df['CS_COD'] + 1e-3)
    df['JS_CS_SW_ratio'] = df['JS_SW'] / (df['CS_SW'] + 1e-3)
    df['HYC_DO_ratio'] = df[f'{type_}_HYC_JS_DO'] / (df[f'{type_}_HYC_DO'] + 1e-3)
    df['CS_MQ_LL_ratio'] = df[f'{type_}_CS_MQ_SSLL'] / (df['CS_LL'] + 1e-3)
    
    return df

train_B = add_ratio_feats(train_B, type_='B')
train_N = add_ratio_feats(train_N, type_='N')
test_B = add_ratio_feats(test_B, type_='B')
test_N = add_ratio_feats(test_N, type_='N')

In [8]:
# target log1p 转换

B_max, B_min = train_B['Label1'].max(), train_B['Label1'].min()
N_max, N_min = train_N['Label2'].max(), train_N['Label2'].min()

train_B['Label1'] = np.log1p(train_B['Label1'])
train_N['Label2'] = np.log1p(train_N['Label2'])

In [9]:
train_B

,time,JS_NH3,CS_NH3,JS_TN,CS_TN,JS_LL,CS_LL,MCCS_NH4,MCCS_NO3,JS_COD,CS_COD,JS_SW,CS_SW,B_HYC_NH4,B_HYC_XD,B_HYC_MLSS,B_HYC_JS_DO,B_HYC_DO,B_CS_MQ_SSLL,B_QY_ORP,Label1,day,hour,dayofweek,JS_CS_NH3_ratio,JS_CS_TN_ratio,JS_CS_LL_ratio,MCCS_NH4_NH3_ratio,MCCS_NO3_NH3_ratio,JS_CS_COD_ratio,JS_CS_SW_ratio,HYC_DO_ratio,CS_MQ_LL_ratio
0,2022-04-30 16:00:00,39.0159,0.0680,52.8394,11.0604,1273.4913,2204.6711,0.0010,8.5796,376.8,7.9,27.8141,29.0165,0.2038,8.3283,6047.2778,0.4141,0.9739,4386.0161,80.9462,7.714169,30,16,5,565.447826,4.776918,0.577633,0.014493,124.342029,47.690166,0.958528,0.424762,1.989419
1,2022-05-02 16:00:00,27.3065,0.0652,40.1798,10.0415,2104.4302,1410.1064,0.1977,8.1337,179.6,8.7,24.5886,25.5326,0.2058,8.2066,6364.8779,0.5817,1.0407,4411.7305,132.8125,7.459690,2,16,0,412.484894,4.000976,1.492390,2.986405,122.865559,20.641306,0.962990,0.558414,3.128649
2,2022-05-03 16:00:00,30.5364,0.0623,46.6933,11.0742,1165.1504,2112.2976,0.1978,8.3301,200.4,9.7,25.4666,26.0163,0.1960,8.9800,6426.9058,0.5075,2.0826,4393.7695,117.2599,7.239251,3,16,1,482.407583,4.216023,0.551603,3.124803,131.597156,20.657664,0.978833,0.243569,2.080089
3,2022-05-04 16:00:00,34.6477,0.0484,52.6615,11.0072,1847.4833,875.0145,0.2086,8.3724,254.7,11.3,25.9736,26.6159,0.2038,8.5756,6174.0825,0.5547,0.7714,4394.1621,105.2517,7.356248,4,16,2,701.370445,4.783843,2.111372,4.222672,169.481781,22.537829,0.975831,0.718151,5.021811
4,2022-05-05 16:00:00,32.1023,0.0277,48.7988,10.4215,1396.4414,1495.3458,0.2059,7.4816,244.2,10.9,26.2821,27.1741,0.1979,7.8180,6123.8320,0.5601,0.8287,4389.8438,97.8733,7.576261,5,16,3,1118.547038,4.682063,0.933858,7.174216,260.682927,22.401615,0.967139,0.675063,2.935669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35063,2022-07-18 02:30:00,27.1314,0.0169,30.2603,9.2619,985.8353,1111.9569,0.1428,9.0365,144.6,9.2,27.3868,31.1426,0.2018,7.1839,6057.0928,0.1099,1.5306,4396.0273,133.7529,9.226712,18,2,0,1515.720670,3.266828,0.886576,7.977654,504.832402,15.715683,0.879372,0.071755,3.953412
35064,2022-07-18 02:32:00,27.1314,0.0169,30.2603,9.2619,1008.5851,1680.4688,0.1425,9.0495,144.6,9.2,27.3911,31.1435,0.1999,8.6522,6098.3140,0.1197,1.4823,4393.5732,133.3188,9.140356,18,2,0,1515.720670,3.266828,0.600180,7.960894,505.558659,15.715683,0.879484,0.080698,2.614491
35065,2022-07-18 02:34:00,27.1314,0.0169,30.2603,9.2619,977.8567,1633.5309,0.1425,9.0419,144.6,9.2,27.3971,31.1448,0.2156,7.7061,6073.9736,0.1424,1.4859,4394.0645,132.9572,9.306930,18,2,0,1515.720670,3.266828,0.598615,7.960894,505.134078,15.715683,0.879640,0.095770,2.689917
35066,2022-07-18 02:36:00,27.1314,0.0169,30.2603,9.2619,974.0818,1247.8890,0.1425,9.0093,144.6,9.2,27.4090,31.1455,0.2117,9.1076,6104.5952,0.1102,1.4230,4390.8252,132.9572,9.224683,18,2,0,1515.720670,3.266828,0.780583,7.960894,503.312849,15.715683,0.880003,0.077388,3.518600


In [10]:
# 分别训练五折 LGB

def run_lgb(df_train, df_test, ycol, n_splits=5, seed=42):
    use_feats = [col for col in df_test.columns if col not in ['time']]
    model = lgb.LGBMRegressor(num_leaves=256,
                              max_depth=16,
                              learning_rate=0.1,
                              n_estimators=10000,
                              subsample=0.8,
                              feature_fraction=0.8,
                              reg_alpha=0.1,
                              reg_lambda=0.1,
                              random_state=seed,
                              metric=None)
    oof = []
    prediction = df_test[['time']]
    prediction[ycol] = 0
    df_importance_list = []
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[use_feats])):
        X_train = df_train.iloc[trn_idx][use_feats]
        Y_train = df_train.iloc[trn_idx][ycol]
        X_val = df_train.iloc[val_idx][use_feats]
        Y_val = df_train.iloc[val_idx][ycol]
        lgb_model = model.fit(X_train,
                              Y_train,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=500,
                              eval_metric='rmse',
                              early_stopping_rounds=50)
        pred_val = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration_)
        df_oof = df_train.iloc[val_idx][['time', ycol]].copy()
        df_oof['pred'] = pred_val
        oof.append(df_oof)
        pred_test = lgb_model.predict(df_test[use_feats], num_iteration=lgb_model.best_iteration_)
        prediction[ycol] += pred_test / n_splits
        df_importance = pd.DataFrame({
            'column': use_feats,
            'importance': lgb_model.feature_importances_,
        })
        df_importance_list.append(df_importance)
        del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()
    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby(['column'])['importance'].agg(
        'mean').sort_values(ascending=False).reset_index()
    display(df_importance.head(50))
    df_oof = pd.concat(oof).reset_index(drop=True)
    df_oof[ycol] = np.expm1(df_oof[ycol])
    df_oof['pred'] = np.expm1(df_oof['pred'])
    prediction[ycol] = np.expm1(prediction[ycol])
    
    return df_oof, prediction

In [11]:
df_oof_B, pred_B = run_lgb(train_B, test_B, ycol='Label1')
df_oof_N, pred_N = run_lgb(train_N, test_N, ycol='Label2')

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


,column,importance
0,B_HYC_DO,3362.4
1,B_HYC_MLSS,2952.4
2,B_QY_ORP,2751.0
3,JS_CS_SW_ratio,2745.2
4,B_CS_MQ_SSLL,2724.4
5,MCCS_NO3,2657.2
6,HYC_DO_ratio,2651.4
7,B_HYC_JS_DO,2585.4
8,MCCS_NH4,2553.2
9,B_HYC_XD,2545.0


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


,column,importance
0,N_HYC_DO,3449.4
1,N_QY_ORP,3052.8
2,N_HYC_MLSS,2884.2
3,N_CS_MQ_SSLL,2847.0
4,HYC_DO_ratio,2834.4
5,N_HYC_NH4,2803.0
6,MCCS_NO3,2773.8
7,JS_LL,2719.4
8,JS_CS_SW_ratio,2705.8
9,N_HYC_JS_DO,2602.6


In [12]:
def calc_score(df1, df2):
    df1['loss1'] = (abs(df1['Label1']**2 - df1['pred']**2))**0.5
    df2['loss2'] = (abs(df2['Label2']**2 - df2['pred']**2))**0.5
    loss = (df1['loss1'].sum()/len(df1) + df2['loss2'].sum()/len(df2)) / 2
    score = (1 / (1 + loss)) * 1000
    return score

calc_score(df_oof_B, df_oof_N)

0.27224269516244864

In [13]:
# df_oof_B['pred'] = np.clip(df_oof_B['pred'], B_min, B_max)
# df_oof_N['pred'] = np.clip(df_oof_N['pred'], N_min, N_max)

# calc_score(df_oof_B, df_oof_N)

In [14]:
# 提交

sub = pd.read_csv('data/sample_submission.csv')
sub['Label1'] = pred_B['Label1'].values
sub['Label2'] = pred_N['Label2'].values
sub

,time,Label1,Label2
0,2022/7/18 2:40,10940.167276,9361.676928
1,2022/7/18 2:42,11283.067512,9553.284032
2,2022/7/18 2:44,11039.583000,9628.834523
3,2022/7/18 2:46,11616.019578,9721.076808
4,2022/7/18 2:48,11512.476873,9885.213705
...,...,...,...
9995,2022/7/31 23:50,11840.073693,12484.667676
9996,2022/7/31 23:52,11594.796396,12772.387555
9997,2022/7/31 23:54,12005.388621,12641.535368
9998,2022/7/31 23:56,11937.976920,12535.487893


In [16]:
sub.to_csv('baseline091301.csv', index=False)